In [1]:
#using conda env mlops_2
import mlflow
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_wine
from sklearn.metrics import accuracy_score
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import tensorflow as tf

#also needed to conda install fsspec
# conda install -c conda-forge huggingface_hub



2025-03-24 22:23:26.243067: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-24 22:23:26.322811: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-24 22:23:26.322886: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-24 22:23:26.326700: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-24 22:23:26.357064: I tensorflow/core/platform/cpu_feature_guar

## preprocessing

In [2]:
#text data
fraud = pd.read_csv("hf://datasets/amitkedia/Financial-Fraud-Dataset/Final_Dataset.csv")

/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
fraud.head()

,Fillings,Fraud
0,nanitem 14 exhibits financial statements repor...,1
1,item 14 principal accounting fees services mat...,0
2,item 14 exhibits financial statements schedule...,1
3,item 14 exhibits financial statement schedules...,1
4,item 14 exhibits financial statement schedules...,0


In [3]:

# Convert the "Fraud" column to binary
fraud["Fraud"] = fraud["Fraud"].map({"yes": 1, "no": 0})

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(fraud["Fillings"], fraud["Fraud"], test_size=0.2, random_state=42)

In [6]:
X_train.head()

69     item 14 principal accounting fees services omi...
138    item 14 principal accounting fees services omi...
2      item 14 exhibits financial statements schedule...
93     item 14 principal accountant fees services omi...
136    item 14 exhibits financial statements schedule...
Name: Fillings, dtype: object

In [5]:

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Perform a simple matrix multiplication on GPU
with tf.device('/GPU:0'):
    a = tf.random.normal([1000, 1000])
    b = tf.random.normal([1000, 1000])
    c = tf.matmul(a, b)

print("Executed on GPU successfully!")


Num GPUs Available:  1
Executed on GPU successfully!


2025-03-24 22:26:07.728498: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-24 22:26:07.945439: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-24 22:26:07.945495: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-24 22:26:07.950473: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-24 22:26:07.950537: I external/local_xla/xla/stream_executor

In [6]:


# Load the Universal Sentence Encoder model
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Function to generate embeddings in batches
def generate_embeddings_in_batches(data, batch_size=8):
    """
    Generates embeddings for a given dataset in batches to optimize memory usage.
    
    Args:
        data (list): List of text inputs.
        batch_size (int): Number of samples per batch.

    Returns:
        np.array: Numpy array containing the generated embeddings.
    """
    embeddings = []
    
    # Check for GPU availability
    device = "/GPU:0" if tf.config.list_physical_devices('GPU') else "/CPU:0"
    print(f"Using device: {device}")

    # Convert data to TensorFlow dataset for efficient batching
    dataset = tf.data.Dataset.from_tensor_slices(data).batch(batch_size)

    # Generate embeddings with appropriate device placement
    with tf.device(device):
        for batch in dataset:
            batch_embeddings = use_model(batch).numpy()
            embeddings.append(batch_embeddings)

    return np.vstack(embeddings)

# Ensure X_train and X_test are converted to lists
X_train_embeddings = generate_embeddings_in_batches(X_train.tolist())
X_test_embeddings = generate_embeddings_in_batches(X_test.tolist())

# Verify output
print("Train Embeddings Shape:", X_train_embeddings.shape)
print("Test Embeddings Shape:", X_test_embeddings.shape)


Using device: /GPU:0
Using device: /GPU:0
Train Embeddings Shape: (136, 512)
Test Embeddings Shape: (34, 512)


In [7]:
X_train_embeddings

array([[ 0.04456453, -0.04456453,  0.04456453, ..., -0.04456453,
        -0.04456453,  0.04456453],
       [-0.04458539, -0.04458539,  0.04458539, ..., -0.04458539,
        -0.04458539,  0.04458539],
       [ 0.04424039, -0.04424039,  0.04424039, ..., -0.04424039,
        -0.04424039,  0.04424039],
       ...,
       [ 0.04424445, -0.04424445,  0.04424445, ..., -0.04424445,
        -0.04424445,  0.04424445],
       [ 0.0443496 , -0.0443496 ,  0.0443496 , ..., -0.0443496 ,
        -0.0443496 ,  0.0443496 ],
       [ 0.04433902, -0.04433902,  0.04433902, ..., -0.04433902,
        -0.04433902,  0.04433902]], dtype=float32)

## model selection
- evaluating models on unseen data
- no data leakage; split and then embedded respectively


In [17]:
mlflow.end_run()

In [8]:


mlflow.set_experiment("fraud_detection")

# Define models to train
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier()
}

# Start MLflow tracking
mlflow.start_run()

for model_name, model in models.items():
    # Train the model using embeddings
    model.fit(X_train_embeddings, y_train)
    
    # Evaluate the model
    accuracy = model.score(X_test_embeddings, y_test)
    
    # Log model and metrics
    mlflow.log_metric(f"{model_name}_accuracy", accuracy)
    mlflow.sklearn.log_model(model, model_name)

    print(f"{model_name}: Accuracy = {accuracy:.4f}")

mlflow.end_run()

Traceback (most recent call last):
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 329, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 427, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 1373, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 1366, in _read_helper
    result = read_yaml(root, file_name)
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/utils/file_utils.py", line 310, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' d

Traceback (most recent call last):
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 329, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 427, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 1373, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 1366, in _read_helper
    result = read_yaml(root, file_name)
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/utils/file_utils.py", line 310, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' d

PermissionError: [Errno 13] Permission denied: '/Users'

In [21]:
# Check embeddings distribution
print("Embeddings shape:", X_train_embeddings.shape)
print("\nSample of unique predictions:")
for model_name, model in models.items():
    preds = model.predict(X_test_embeddings)
    print(f"{model_name} unique predictions:", np.unique(preds, return_counts=True))

# Check for potential data leakage or preprocessing issues
print("\nLabel distribution:")
print("Train:", np.unique(y_train, return_counts=True))
print("Test:", np.unique(y_test, return_counts=True))

# Check embedding values
print("\nEmbedding statistics:")
print("Mean:", np.mean(X_train_embeddings))
print("Std:", np.std(X_train_embeddings))
print("Min:", np.min(X_train_embeddings))
print("Max:", np.max(X_train_embeddings))

Embeddings shape: (136, 512)

Sample of unique predictions:
Logistic Regression unique predictions: (array([0, 1]), array([16, 18]))
Random Forest unique predictions: (array([0, 1]), array([17, 17]))
XGBoost unique predictions: (array([0, 1]), array([16, 18]))

Label distribution:
Train: (array([0, 1]), array([67, 69]))
Test: (array([0, 1]), array([18, 16]))

Embedding statistics:
Mean: 0.0010253671
Std: 0.044182274
Min: -0.05049441
Max: 0.05049441


## tuning

In [31]:
# # # hyper param tuning for xgboost with baysian optimization using hyperopt

# # Define the objective function
# def objective(params):
#     # Create the model with the suggested hyperparameters
#     model = xgb.XGBClassifier(
#         max_depth=int(params['max_depth']),
#         min_child_weight=int(params['min_child_weight']),
#         gamma=params['gamma'],
#         subsample=params['subsample'],
#         colsample_bytree=params['colsample_bytree'],
#         learning_rate=params['learning_rate'],
#         use_label_encoder=False,
#         eval_metric='logloss'
#     )

#     # Fit the model
#     model.fit(X_train_embeddings, y_train)

#     # Predict and evaluate
#     preds = model.predict(X_test_embeddings)
#     accuracy = accuracy_score(y_test, preds)

#     # Return the accuracy as the objective to minimize (negative because we want to maximize accuracy)
#     return {'loss': -accuracy, 'status': STATUS_OK}

# # Define the hyperparameter space
# space = {
#     'max_depth': hp.randint('max_depth', 3, 10),  # Integer values from 3 to 9
#     'min_child_weight': hp.randint('min_child_weight', 1, 6),  # Integer values from 1 to 5
#     'gamma': hp.uniform('gamma', 0, 0.5),  # Continuous values from 0 to 0.5
#     'subsample': hp.uniform('subsample', 0.5, 1.0),  # Continuous values from 0.5 to 1.0
#     'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),  # Continuous values from 0.5 to 1.0
#     'learning_rate': hp.uniform('learning_rate', 0.01, 0.2)  # Continuous values from 0.01 to 0.2
# }

# # Create a Trials object to keep track of the results
# trials = Trials()

# # Run Hyperopt
# best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

# print("Best Hyperparameters:", best)
# import mlflow
# import mlflow.xgboost
# from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
# from sklearn.metrics import accuracy_score
# import xgboost as xgb

# Set the experiment name
mlflow.set_experiment("XGBoost_Tuning")

# Enable autologging for XGBoost models
mlflow.xgboost.autolog()

# Define the objective function
def objective(params):
    with mlflow.start_run(nested=True):  # Start an MLflow run
        # Create the model with the suggested hyperparameters
        model = xgb.XGBClassifier(
            max_depth=int(params['max_depth']),
            min_child_weight=int(params['min_child_weight']),
            gamma=params['gamma'],
            subsample=params['subsample'],
            colsample_bytree=params['colsample_bytree'],
            learning_rate=params['learning_rate'],
            use_label_encoder=False,
            eval_metric='logloss'
        )

        # Fit the model
        model.fit(X_train_embeddings, y_train)

        # Predict and evaluate
        preds = model.predict(X_test_embeddings)
        accuracy = accuracy_score(y_test, preds)

        # MLflow autologging will log parameters and metrics automatically
        return {'loss': -accuracy, 'status': STATUS_OK}

# Define the hyperparameter space
space = {
    'max_depth': hp.randint('max_depth', 3, 10),
    'min_child_weight': hp.randint('min_child_weight', 1, 6),
    'gamma': hp.uniform('gamma', 0, 0.5),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2)
}

# Create a Trials object to keep track of the results
trials = Trials()

# Run Hyperopt
with mlflow.start_run(run_name="Hyperopt_XGBoost"):  # Top-level MLflow run
    best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

print("Best Hyperparameters:", best)


2025/03/24 22:17:54 INFO mlflow.tracking.fluent: Experiment with name 'XGBoost_Tuning' does not exist. Creating a new experiment.


  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:17:55] WARNING: /croot/xgboost-split_1724073744422/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)

2025/03/24 22:18:04 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:18:04] WARNING: /croot/xgboost-split_1724073744422/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2025/03/24 22:18:07 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmppby77820/model, flavor: xgboost). Fall back to return ['xgboost==2.1.1']. Set logging level to DEBUG to see the full traceback. 

2025/03/24 22:18:07 WARNING mlflow.utils.requirem

  2%|▏         | 1/50 [00:12<10:35, 12.97s/trial, best loss: -0.7352941176470589]

/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:18:07] WARNING: /croot/xgboost-split_1724073744422/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)

2025/03/24 22:18:14 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:18:14] WARNING: /croot/xgboost-split_1724073744422/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2025/03/24 22:18:16 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpi1bf0h7o/model, flavor: xgboost). Fall back to return ['xgboost==2.1.1']. Set logging level to DEBUG to see the full traceback. 

2025/03/24 22:18:16 WARNING mlflow.utils.requirem

...

/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:21:08] WARNING: /croot/xgboost-split_1724073744422/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)

2025/03/24 22:21:14 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:21:14] WARNING: /croot/xgboost-split_1724073744422/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2025/03/24 22:21:16 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpze1h8uv3/model, flavor: xgboost). Fall back to return ['xgboost==2.1.1']. Set logging level to DEBUG to see the full traceback. 

2025/03/24 22:21:16 WARNING mlflow.utils.requirem

 42%|████▏     | 21/50 [03:22<04:58, 10.30s/trial, best loss: -0.8529411764705882]

/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:21:17] WARNING: /croot/xgboost-split_1724073744422/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)

2025/03/24 22:21:27 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:21:27] WARNING: /croot/xgboost-split_1724073744422/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."



 42%|████▏     | 21/50 [03:34<04:56, 10.22s/trial, best loss: -0.8529411764705882]


KeyboardInterrupt: 

100%|██████████| 50/50 [05:43<00:00,  6.87s/trial, best loss: -0.8823529411764706]
Best Hyperparameters: {'colsample_bytree': 0.8820986618419748, 'gamma': 0.25925063074647053, 'learning_rate': 0.16812368903322059, 'max_depth': 6, 'min_child_weight': 2, 'subsample': 0.6286518997410409}



In [11]:
#randon forest tuning

import mlflow
import mlflow.sklearn
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Set the experiment name
mlflow.set_experiment("RandomForest_Tuning")

# Enable autologging for sklearn models
mlflow.sklearn.autolog()

# Define the objective function
def objective(params):
    with mlflow.start_run(nested=True):  # Start an MLflow run
        # Create the model with the suggested hyperparameters
        model = RandomForestClassifier(
            n_estimators=int(params['n_estimators']),
            max_depth=int(params['max_depth']) if params['max_depth'] is not None else None,
            min_samples_split=int(params['min_samples_split']),
            min_samples_leaf=int(params['min_samples_leaf']),
            max_features=params['max_features'],
            bootstrap=params['bootstrap'],
            random_state=42,  # Ensuring reproducibility
            n_jobs=-1  # Use all cores
        )

        # Fit the model
        model.fit(X_train_embeddings, y_train)

        # Predict and evaluate
        preds = model.predict(X_test_embeddings)
        accuracy = accuracy_score(y_test, preds)

        # MLflow autologging takes care of logging parameters and metrics automatically
        return {'loss': -accuracy, 'status': STATUS_OK}

# Define the hyperparameter space for Random Forest
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 10),  # Trees between 50 and 500 (step of 10)
    'max_depth': hp.quniform('max_depth', 5, 30, 1),  # Depth between 5 and 30
    'min_samples_split': hp.quniform('min_samples_split', 2, 20, 1),  # Min samples to split a node (2 to 20)
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 10, 1),  # Min samples per leaf (1 to 10)
    'max_features': hp.choice('max_features', ['sqrt', 'log2', None]),  # Use 'sqrt', 'log2', or None
    'bootstrap': hp.choice('bootstrap', [True, False])  # Whether to use bootstrap sampling
}

# Create a Trials object to keep track of the results
trials = Trials()

# Run Hyperopt
with mlflow.start_run(run_name="Hyperopt_RandomForest"):  # Top-level MLflow run
    best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

print("Best Hyperparameters:", best)



Traceback (most recent call last):
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 329, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 427, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 1373, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 1366, in _read_helper
    result = read_yaml(root, file_name)
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/utils/file_utils.py", line 310, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' d

Traceback (most recent call last):
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 329, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 427, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 1373, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 1366, in _read_helper
    result = read_yaml(root, file_name)
  File "/home/kdb/miniconda3/envs/mlops_2/lib/python3.9/site-packages/mlflow/utils/file_utils.py", line 310, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' d

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

2025/03/24 22:30:15 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'



  2%|▏         | 1/50 [00:06<05:15,  6.45s/trial, best loss: -0.7352941176470589]

...

2025/03/24 22:30:40 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'



 12%|█▏        | 6/50 [00:30<03:43,  5.09s/trial, best loss: -0.8823529411764706]

2025/03/24 22:30:46 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'



 14%|█▍        | 7/50 [00:35<03:32,  4.93s/trial, best loss: -0.8823529411764706]

2025/03/24 22:30:50 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'



In [10]:
mlflow.end_run()